In [1]:
import pandas as pd
import json
import urllib
import pandas as pd
import io
import math
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from collections import Counter, defaultdict
import spacy

In [2]:
class LSTM:

    def __init__(self,Xtrain, Xvalidate, Xtest, ytrain, yvalidate, ytest, mute = False):
        print("-"*60)
        print(f"Initializing LSTM Classifier")
        


        self.Xtrain = Xtrain
        self.ytrain = ytrain
        self.Xvalidate = Xvalidate
        self.yvalidate = yvalidate
        self.Xtest = Xtest
        self.ytest = ytest

        self.mute = mute

        self.prepro()

        # Classification
        self.classify()

        # Evalaution
        self.evaluation()

    def preprocess(self):
        print("-"*60)
        print(f"Preprocessing")
            
        # The maximum number of words to be used. (most frequent)
        MAX_NB_WORDS = 2000
        # Max number of words in each message.
        MAX_SEQUENCE_LENGTH = 250
        # This is fixed.
        EMBEDDING_DIM = 100
        tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
        tokenizer.fit_on_texts(df['messages'].values)
        word_index = tokenizer.word_index
        print(f'Found {len(word_index)} unique tokens.')


        X = tokenizer.texts_to_sequences(df['messages'].values)

        X = tokenizer.sequences_to_matrix(X, mode= "tfidf")

        column_to_dummies = [
            'speakers',
            'receivers',
            'seasons',
            'receiver_labels'
            
        ]
        # https://stackoverflow.com/questions/23208745/adding-dummy-columns-to-the-original-dataframe

        df = pd.concat([df, pd.get_dummies(df[column_to_dummies])], axis=1)
        column_to_dummies += ["messages"]
        df = df.drop(columns=column_to_dummies)
        df['sender_labels'] = df.sender_labels.astype(int)
        df.rename(columns = {"sender_labels": "class"}, inplace=True)

        df = pd.concat([df, pd.DataFrame(X)], axis=1)

        return df

    def train(self):
        self.model = Sequential()
        self.model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
        self.model.add(SpatialDropout1D(0.2))
        self.model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

        # Last Layer with 2 Nodes
        self.model.add(Dense(2, activation='softmax'))
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['f1'])

        epochs = 5
        batch_size = 64


        history = self.model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



In [3]:
df_train = pd.read_csv('train.csv')


def data_processor(df):
    # The maximum number of words to be used. (most frequent)
    MAX_NB_WORDS = 2000
    # Max number of words in each message.
    MAX_SEQUENCE_LENGTH = 250
    # This is fixed.
    EMBEDDING_DIM = 100
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(df['messages'].values)
    word_index = tokenizer.word_index
    print(f'Found {len(word_index)} unique tokens.')


    #pd.DataFrame(X.numpy())

    X = tokenizer.texts_to_sequences(df['messages'].values)

    X = tokenizer.sequences_to_matrix(X, mode= "tfidf")

    column_to_dummies = [
        'speakers',
        'receivers',
        'seasons',
        'receiver_labels'
        
    ]
    # https://stackoverflow.com/questions/23208745/adding-dummy-columns-to-the-original-dataframe

    df = pd.concat([df, pd.get_dummies(df[column_to_dummies])], axis=1)
    column_to_dummies += ["messages"]
    df = df.drop(columns=column_to_dummies)
    df['sender_labels'] = df.sender_labels.astype(int)
    df.rename(columns = {"sender_labels": "class"}, inplace=True)

    df = pd.concat([df, pd.DataFrame(X)], axis=1)

    return df

In [5]:
df = data_processor(df_train)

Found 8884 unique tokens.


In [10]:
df

,class,absolute_message_index,relative_message_index,years,game_score,game_score_delta,game,speakers_austria,speakers_england,speakers_france,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1,0.0,0.0,1901,3,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,1901,3,0,10,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,1901,3,0,5,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,1901,3,0,2,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,1901,4,1,3,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13127,1,3297.0,200.0,1910,7,1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13128,1,3298.0,201.0,1910,6,-1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13129,1,3299.0,202.0,1910,6,-1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13130,1,3300.0,203.0,1910,6,-1,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d30aab93-185d-4715-858f-78b8c83c5d5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>